In [1]:
import pandas as pd
pd.options.display.max_rows=300

import numpy as np

# MIMIC

In [2]:
df_time = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/00.data_mimic//disease_definition/AKI/sk_event_time.csv")
df_time = df_time[["stay_id", "saki_onset"]]
df_time["saki_onset"] = pd.to_datetime(df_time["saki_onset"])


In [3]:
df_type = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/01.MIMICIV_SAKI_trajCluster/df_mixAK_fea4_C3.csv")
df_type_filt = df_type.loc[:,["stay_id","groupHPD"]]
df_type_filt = df_type_filt.drop_duplicates()
all_lst = df_type_filt.stay_id.unique().tolist()
print("纳入患者数量：",len(all_lst))

纳入患者数量： 4713


In [4]:
df = pd.read_csv("./mimic_sofa.csv")
df = pd.merge(df,df_type_filt, how="inner",on="stay_id") #df[df["stay_id"].isin(all_lst)]
df = df[['stay_id','groupHPD', 'starttime', 'endtime','respiration_24hours', 'coagulation_24hours', 'liver_24hours',
       'cardiovascular_24hours', 'cns_24hours', 'renal_24hours','sofa_24hours']]
df = df.rename(columns={'respiration_24hours':"respiration_sofa", 'coagulation_24hours':"coagulation_sofa", 'liver_24hours': 'liver_sofa',
       'cardiovascular_24hours':"cardiovascular_sofa", 'cns_24hours':'cns_sofa', 'renal_24hours':'renal_sofa','sofa_24hours':"sofa"})
df["starttime"] = pd.to_datetime(df["starttime"], format='%d/%m/%Y %H:%M:%S')
df["endtime"] = pd.to_datetime(df["endtime"], format='%d/%m/%Y %H:%M:%S')
df.head(2)

,stay_id,groupHPD,starttime,endtime,respiration_sofa,coagulation_sofa,liver_sofa,cardiovascular_sofa,cns_sofa,renal_sofa,sofa
0,30003598,1,2189-04-14 20:00:00,2189-04-14 21:00:00,2,0,0,1,0,0,3
1,30003598,1,2189-04-14 21:00:00,2189-04-14 22:00:00,2,0,0,1,0,1,4


In [5]:
df_merge = pd.merge(df, df_time, how="inner", on="stay_id")
df_merge["time"] = (df_merge["starttime"] - df_merge["saki_onset"]).map(lambda x:x/np.timedelta64(6, 'h')).tolist()
df_merge["time"] = df_merge["time"].map(lambda x:x//1) # -1指sepsis前6小时的数据，0指sepsis后第一个6小时的数据
df_merge["time"] = df_merge["time"].map(lambda x: x+1 if x>=0 else x)
df_merge = df_merge[(df_merge["time"]>=-4) &(df_merge["time"]<=28)]
df_merge = df_merge.groupby(["stay_id","time"]).agg('max').reset_index()
df_merge = df_merge.drop(["starttime", "endtime", "saki_onset"],axis=1)
len(df_merge.stay_id.unique())

4713

In [6]:
df = df_merge
com_lst = []
for i in df.stay_id.unique().tolist():
    min_t = -2  # int(min(df_fea[df_fea.stay_id==i].time))
    max_t = int(max(df[df.stay_id==i].time))
    for j in range(min_t,max_t+1,1):
        com_lst.append([i,j])
df_complete = pd.DataFrame(com_lst,columns=["stay_id", "time"])  

df_fea = pd.merge(df_complete, df_merge, how="left", on=["stay_id","time"])
df_fea = df_fea[df_fea["time"]!=0]
df_fea = df_fea.sort_values(["stay_id","time"])

tmp_df = pd.DataFrame()
for i,j in df_fea.groupby(["stay_id"]):
    j = j.fillna(method="ffill")
    j = j.fillna(method="bfill")
    tmp_df = pd.concat([tmp_df, j], axis=0)
df_im = tmp_df
df_im["sofa"] = df_im["respiration_sofa"] + df_im["coagulation_sofa"] + df_im["liver_sofa"] + df_im["cardiovascular_sofa"] + df_im["cns_sofa"] + df_im["renal_sofa"] 
df_im.to_csv("mimic_sofa_clean.csv",index=False)
df_im.head(2)

,stay_id,time,groupHPD,respiration_sofa,coagulation_sofa,liver_sofa,cardiovascular_sofa,cns_sofa,renal_sofa,sofa
0,30003598,-2,1.0,2.0,0.0,0.0,1.0,0.0,1.0,4.0
1,30003598,-1,1.0,2.0,0.0,0.0,1.0,0.0,1.0,4.0


# AUMCdb

In [7]:
df_time = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3//00.data_aumc/disease_definition/AKI/aumcdb_sk_event_time.csv")
df_time = df_time[['stay_id','saki_onset']]
df_time.head(1)

,stay_id,saki_onset
0,0,NaN


In [8]:
df_type = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3//02.AUMCdb_SAKI_trajCluster/df_mixAK_fea3_C3_aumc.csv")
df_type_filt = df_type.loc[:,["stay_id","groupHPD"]]
df_type_filt = df_type_filt.drop_duplicates()
all_lst = df_type_filt.stay_id.unique().tolist()
print("纳入患者数量：",len(all_lst))

纳入患者数量： 2183


In [9]:
df = pd.read_csv("./aumcdb_sofa.csv") # hour
df[(df["sofa_total_score"]!=0) & (df["admissionid"]==5)]


,admissionid,time,sofa_respiration_score,sofa_coagulation_score,sofa_liver_score,sofa_cardiovascular_score,sofa_cns_score,sofa_renal_score,sofa_total_score
165,5,0.0,NaN,0.0,0.0,4.0,0.0,0.0,4.0
166,5,1.0,0.0,NaN,NaN,3.0,NaN,0.0,3.0
167,5,2.0,1.0,NaN,NaN,1.0,NaN,0.0,2.0
168,5,3.0,NaN,NaN,NaN,1.0,NaN,0.0,1.0
169,5,4.0,NaN,NaN,NaN,1.0,NaN,0.0,1.0
170,5,5.0,NaN,NaN,NaN,1.0,NaN,0.0,1.0
171,5,6.0,NaN,NaN,NaN,1.0,NaN,0.0,1.0
172,5,7.0,1.0,NaN,NaN,1.0,NaN,0.0,2.0
173,5,8.0,NaN,NaN,NaN,1.0,NaN,0.0,1.0
174,5,9.0,NaN,NaN,NaN,1.0,0.0,0.0,1.0


In [10]:
df = pd.read_csv("./aumcdb_sofa.csv") # hour
df = df.rename(columns={'admissionid':'stay_id','time':'hr','sofa_respiration_score':"respiration_sofa", 'sofa_coagulation_score':"coagulation_sofa", 'sofa_liver_score': 'liver_sofa',
        'sofa_cardiovascular_score':"cardiovascular_sofa", 'sofa_cns_score':'cns_sofa', 'sofa_renal_score':'renal_sofa','sofa_total_score':"sofa"})
df = pd.merge(df,df_type_filt, how="inner",on="stay_id") #df[df["stay_id"].isin(all_lst)]
df.head(2)

,stay_id,hr,respiration_sofa,coagulation_sofa,liver_sofa,cardiovascular_sofa,cns_sofa,renal_sofa,sofa,groupHPD
0,5,-1.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,2
1,5,0.0,NaN,0.0,0.0,4.0,0.0,0.0,4.0,2


In [11]:
df_merge = pd.merge(df, df_time, how="inner", on="stay_id")
df_merge["hr"] = df_merge["hr"].map(lambda x: x+1 if x>=0 else x)

df_merge["time"] = (df_merge["hr"] - df_merge["saki_onset"])/6
df_merge["time"] = df_merge["time"].map(lambda x:x//1) # -1指sepsis前6小时的数据，0指sepsis后第一个6小时的数据

df_merge["time"] = df_merge["time"].map(lambda x: x+1 if x>=0 else x)
df_merge = df_merge[(df_merge["time"]>=-4) &(df_merge["time"]<=28)]
df_merge = df_merge.groupby(["stay_id","time"]).agg('max').reset_index()
df_merge = df_merge.drop(["hr", "saki_onset"],axis=1)
len(df_merge.stay_id.unique())

2183

In [12]:
df = df_merge
com_lst = []
for i in df.stay_id.unique().tolist():
    min_t = -2 
    max_t = int(max(df[df.stay_id==i].time))
    for j in range(min_t,max_t+1,1):
        com_lst.append([i,j])
df_complete = pd.DataFrame(com_lst,columns=["stay_id", "time"])  

df_fea = pd.merge(df_complete, df_merge, how="left", on=["stay_id","time"])
df_fea = df_fea[df_fea["time"]!=0]
df_fea = df_fea.sort_values(["stay_id","time"])

tmp_df = pd.DataFrame()
for i,j in df_fea.groupby(["stay_id"]):
    j = j.fillna(method="ffill")
    j = j.fillna(method="bfill")
    tmp_df = pd.concat([tmp_df, j], axis=0)
df_im = tmp_df

# # 多重插补
# import miceforest as mf
# kernel = mf.ImputationKernel(
#     df_im,
#     datasets=3,
#     save_all_iterations=True,
#     random_state=10
# )
# kernel.mice(iterations = 3, n_jobs=-1)
# df_im2 = kernel.complete_data(dataset=1)
df_im2 = df_im.fillna(0)
df_im2["sofa"] = df_im2["respiration_sofa"] + df_im2["coagulation_sofa"] + df_im2["liver_sofa"] + df_im2["cardiovascular_sofa"] + df_im2["cns_sofa"] + df_im2["renal_sofa"] 
df_im2.to_csv("aumc_sofa_clean.csv",index=False)
len(df_im2.stay_id.unique())

2183

# eICU

In [13]:
df_time = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3//00.data_eicu/disease_definition/AKI/eicu_saki_event_time.csv")
df_time = df_time[['stay_id','saki_onset']]
df_time.head(1)

,stay_id,saki_onset
0,224606,NaN


In [14]:
df_type = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/03.eICU_SAKI_trajCluster/df_mixAK_fea4_C3_eicu.csv")
df_type_filt = df_type.loc[:,["stay_id","groupHPD"]]
df_type_filt = df_type_filt.drop_duplicates()
all_lst = df_type_filt.stay_id.unique().tolist()
print("纳入患者数量：",len(all_lst))

纳入患者数量： 1417


In [15]:
df = pd.read_csv("./eicu_sofa.csv") # hour
df = df[['patientunitstayid', 'hr','respiration_24hours', 'coagulation_24hours', 'liver_24hours',
       'cardiovascular_24hours', 'cns_24hours', 'renal_24hours',
       'sofa_24hours']]
df = df.rename(columns={'patientunitstayid':'stay_id','respiration_24hours':"respiration_sofa", 'coagulation_24hours':"coagulation_sofa", 'liver_24hours': 'liver_sofa',
       'cardiovascular_24hours':"cardiovascular_sofa", 'cns_24hours':'cns_sofa', 'renal_24hours':'renal_sofa','sofa_24hours':"sofa"})
df = pd.merge(df,df_type_filt, how="inner",on="stay_id") 
df.head(2)

,stay_id,hr,respiration_sofa,coagulation_sofa,liver_sofa,cardiovascular_sofa,cns_sofa,renal_sofa,sofa,groupHPD
0,243208,0,0,0,0,0,0,1,1,2
1,243208,1,0,0,0,1,0,1,2,2


In [16]:
df_merge = pd.merge(df, df_time, how="inner", on="stay_id")
df_merge["hr"] = df_merge["hr"].map(lambda x: x+1 if x>=0 else x)

df_merge["time"] = (df_merge["hr"] - df_merge["saki_onset"])/6
df_merge["time"] = df_merge["time"].map(lambda x:x//1) # -1指sepsis前6小时的数据，0指sepsis后第一个6小时的数据

df_merge["time"] = df_merge["time"].map(lambda x: x+1 if x>=0 else x)
df_merge = df_merge[(df_merge["time"]>=-4) &(df_merge["time"]<=28)]
df_merge = df_merge.groupby(["stay_id","time"]).agg('max').reset_index()
df_merge = df_merge.drop(["hr", "saki_onset"],axis=1)


In [17]:
df = df_merge
com_lst = []
for i in df.stay_id.unique().tolist():
    min_t = -2  
    max_t = int(max(df[df.stay_id==i].time))
    for j in range(min_t,max_t+1,1):
        com_lst.append([i,j])
df_complete = pd.DataFrame(com_lst,columns=["stay_id", "time"])  

df_fea = pd.merge(df_complete, df_merge, how="left", on=["stay_id","time"])
df_fea = df_fea[df_fea["time"]!=0]
df_fea = df_fea.sort_values(["stay_id","time"])

tmp_df = pd.DataFrame()
for i,j in df_fea.groupby(["stay_id"]):
    j = j.fillna(method="ffill")
    j = j.fillna(method="bfill")
    tmp_df = pd.concat([tmp_df, j], axis=0)
df_im = tmp_df

# 多重插补
# import miceforest as mf
# kernel = mf.ImputationKernel(
#     df_im,
#     datasets=3,
#     save_all_iterations=True,
#     random_state=10
# )
# kernel.mice(iterations = 3, n_jobs=-1)
# df_im2 = kernel.complete_data(dataset=1)
df_im2 = df_im.fillna(0)

df_im2["sofa"] = df_im2["respiration_sofa"] + df_im2["coagulation_sofa"] + df_im2["liver_sofa"] + df_im2["cardiovascular_sofa"] + df_im2["cns_sofa"] + df_im2["renal_sofa"] 

df_im2.to_csv("eicu_sofa_clean.csv",index=False)

# merge

In [18]:
df1 = pd.read_csv("./eicu_sofa_clean.csv")
df1["dataset"] = "eicu"
df2 = pd.read_csv("./aumc_sofa_clean.csv")
df2["dataset"] = "aumcdb"
df3 = pd.read_csv("./mimic_sofa_clean.csv")
df3["dataset"] = "mimic"

In [19]:
df_all = pd.concat([df1,df2,df3],axis=0)
df_all.to_csv("df_sofa_merge.csv",index=False)

In [20]:
df_all.dataset.unique()

array(['eicu', 'aumcdb', 'mimic'], dtype=object)